UFC Fight Prediction Using Random Forest

Overview

This project builds a UFC fight prediction model using the Random Forest Classifier. The workflow involves data cleaning, feature engineering, hyperparameter tuning, training the model, and making predictions on new fights.

1. Load and Clean Data

Read the dataset

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import accuracy_score

In [ ]:
# get the file path
def get_file_path(folder, file_name):
    return os.path.dirname(os.getcwd()) + "\\" + folder + "\\" + file_name

# read json file
def read_file(folder="data\\transformed", file_name=None):
    return pd.read_csv(get_file_path(folder, file_name))

training_data = read_file(file_name="training_data.csv")
training_data

1. Handle missing values and remove unnecessary columns and  Filter out non-relevant results

In [ ]:
# Drop N/A values
training_data = training_data.dropna(subset=["fighter_name_a", "fighter_name_b","weight_class","event_date_a","event_date_b"])

training_data["Losses_a"] = training_data.Losses_a.fillna(0)

# Drop columns that are not features
non_feature_columns = [
    "fighter_name_a",
    "fighter_name_b",
    "event_date_a",
    "event_date_b",
]


# Remove any draw or nc from the result column
training_data = training_data[~training_data.result.str.contains("draw")]
training_data = training_data[~training_data.result.str.contains("nc")]

training_data = training_data.drop_duplicates()

# drop weight class null 
training_data = training_data.dropna(subset=["weight_class"])

training_data.fillna(0, inplace=True)



clean_training_data = training_data.copy()

clean_training_data.shape

2. Feature Engineering

Convert categorical features into numerical values


In [ ]:
# transform weight class to numerical values
weight_class_encoder = LabelEncoder()
weight_class_encoder.fit(clean_training_data["weight_class"])

clean_training_data["weight_class"] = weight_class_encoder.transform(
    clean_training_data["weight_class"]
)

In [ ]:
# split features and target (results)
X = clean_training_data.drop(columns=non_feature_columns)
X = X.drop(columns=["result"])
y = clean_training_data["result"]

Split features and target variable

In [ ]:
# encode the target
le = LabelEncoder()
y = le.fit_transform(y)

# split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=42
)

3. Train Model

Split data and tune hyperparameters using RandomizedSearchCV

In [ ]:
param_dist = {
    "n_estimators": randint(100, 500),  # More trees improve stability
    "max_depth": randint(5, 30),  # Prevent overly deep trees
    "min_samples_split": randint(2, 20),  # Avoid overfitting small splits
    "max_features": ["sqrt", "log2", None],  # Control number of features per split
}

random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    param_distributions=param_dist,
    n_iter=50,  # Increase if you have time for better tuning
    scoring="accuracy",  # We use accuracy as our metric
    random_state=42,
    n_jobs=-1,  # Use all available CPUs for efficiency
)

random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best Accuracy Score:", random_search.best_score_)

Train final model with optimal parameters and evalute accuracy

In [ ]:
# train the model
model = RandomForestClassifier(
    max_depth=7,
    max_features=None,
    min_samples_split=7,
    n_estimators=236,
    random_state=42,
)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Feature Importance Visualization

In [ ]:
# Visualize the model the most important features
import matplotlib.pyplot as plt
import seaborn as sns

feature_importances = model.feature_importances_
features = X.columns
df = pd.DataFrame({"features": features, "importance": feature_importances})
df = df.sort_values(by="importance", ascending=False)

plt.figure(figsize=(16, 10))
sns.barplot(x=df["importance"], y=df["features"])
plt.title("Feature Importance")
plt.show()

5. Match Prediction

Prepare fighter match data

In [ ]:
def df_selection(df):

    # Delete result column
    df = df.drop(columns=["result"])

    # find all columns ending with a and b
    fighter_columns = [col for col in df.columns if col.endswith("_a")]
    opponent_columns = [col for col in df.columns if col.endswith("_b")]

    # select all columns ending with a and b
    df_a, df_b = df[fighter_columns], df[opponent_columns]

    # remove the suffixes
    df_a.columns = [col[:-2] if col.endswith("_a") else col for col in df_a.columns]
    df_b.columns = [col[:-2] if col.endswith("_b") else col for col in df_b.columns]

    df = pd.concat([df_a, df_b], axis=1)

    # remove duplicate column & order by event date
    df = df.loc[:, ~df.columns.duplicated()]
    df = df.sort_values(by="event_date", ascending=False)

    # get most recent per fighter
    df = df.drop_duplicates(subset=["fighter_name"], keep="first")

    return df

In [ ]:
selection_df = df_selection(training_data)


Create fight matchup features

In [ ]:
def create_match(
    selection_df: pd.DataFrame,
    fighter_a: str | list,
    fighter_b: str | list,
    weight_class: str | list
) -> pd.DataFrame:

    if isinstance(fighter_a, str):
        fighter_a = [fighter_a]
        fighter_b = [fighter_b]
        weight_class = [weight_class]

    if len(fighter_a) != len(fighter_b) or len(fighter_a) != len(weight_class):
        raise ValueError("The number of fighters and weight_classes must be the same")

    # Remove event date from the selection_df
    selection_df = selection_df.drop(columns=["event_date"])

    # get the most recent data for each fighter
    fighter_a_data = selection_df.set_index("fighter_name").loc[fighter_a].reset_index()
    fighter_b_data = selection_df.set_index("fighter_name").loc[fighter_b].reset_index()

    # rename the columns add suffixes

    fighter_a_data.columns = [f"{col}_a" for col in fighter_a_data.columns]
    fighter_b_data.columns = [f"{col}_b" for col in fighter_b_data.columns]

    # reset the index
    fighter_a_data = fighter_a_data.reset_index(drop=True)
    fighter_b_data = fighter_b_data.reset_index(drop=True)

    # merge on index
    features = pd.concat([fighter_a_data, fighter_b_data], axis=1)

    # weight class to series
    features["weight_class"] = pd.Series(weight_class)

    # transform weight class to numerical values
    features["weight_class"] = weight_class_encoder.transform(features["weight_class"])

    # weight class as first column
    features = features[
        ["weight_class"] + [col for col in features.columns if col != "weight_class"]
    ]

    return features

Predict fight outcomes

In [ ]:
# Function to predict the outcome of a fight


def predict_outcome(training_data, features: pd.DataFrame) -> pd.DataFrame:

    # train model on all the data
    model = RandomForestClassifier(
        max_depth=7, min_samples_split=7, n_estimators=236, random_state=42
    )

    X, y = clean_training_data.drop(columns=non_feature_columns), training_data["result"]

    X = X.drop(columns=["result"])
    y = le.fit_transform(y)

    model.fit(X, y)

    # remove fighter_name
    x = features.drop(columns=["fighter_name_a", "fighter_name_b"])

    # change x sequence to match the training data
    x = x[X.columns]

    # make predictions
    y_pred = model.predict(x)

    # add probability of the winning class
    y_pred_proba = model.predict_proba(x)
    y_pred_proba = y_pred_proba.max(axis=1)

    # if b than loss else win
    y_pred = ["loss" if result == 1 else "win" for result in y_pred]

    predictions_df = pd.DataFrame(
        {
            "Fighter": features["fighter_name_a"].values,
            "Opponent": features["fighter_name_b"].values,
            # original y values
            "Result": y_pred,
            "Vote Share": y_pred_proba,
        }
    )

    return predictions_df


features = create_match(
    selection_df,
    fighter_a=[
        "Alex Pereira",
        "Jon Jones",
        "Islam Makhachev",
        "Alexander Volkanovski",
        "Belal Muhammad",
        "Paddy Pimblett",
        "Khamzat Chimaev",
    ],
    fighter_b=[
        "Magomed Ankalaev",
        "Tom Aspinall",
        "Ilia Topuria",
        "Diego Lopes",
        "Jack Della Maddalena",
        "Michael Chandler",
        "Dricus Du Plessis",
    ],
    weight_class=[
        "Light Heavyweight",
        "Heavyweight",
        "Lightweight",
        "Featherweight",
        "Welterweight",
        "Lightweight",
        "Middleweight",
    ],
)


outcome = predict_outcome(training_data, features)

outcome